# GHCN data
Reading in and working with GHCN (GLobal Historical Climate Network) data

see https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/global-historical-climatology-network-ghcn

reading in stations

In [31]:
from ftplib import FTP
from io import StringIO
from contextlib import closing
import pandas as pd

In [33]:
ghcn_specs = [
    {
        "name": "stations",
        "file_name": "ghcnd-stations.txt",
        "desc": "stations",
        "field_specs": [
            {
                "field_name": "ID",
                "pos_from": 1,
                "pos_to": 11
            },
            {
                "field_name": "LATITUDE",
                "pos_from": 13,
                "pos_to": 20
            },
            {
                "field_name": "LONGITUDE",
                "pos_from": 22,
                "pos_to": 30
            },
            {
                "field_name": "ELEVATION",
                "pos_from": 32,
                "pos_to": 37
            },
            {
                "field_name": "STATE",
                "pos_from": 39,
                "pos_to": 40
            },
            {
                "field_name": "NAME",
                "pos_from": 42,
                "pos_to": 71
            },
            {
                "field_name": "GSN FLAG",
                "pos_from": 73,
                "pos_to": 75
            },
            {
                "field_name": "HCN/CRN FLAG",
                "pos_from": 77,
                "pos_to": 79
            },
            {
                "field_name": "WMO ID",
                "pos_from": 81,
                "pos_to": 85
            },
        ]
    },
    {
        "name": "readme",
        "file_name": "readme.txt",
        "desc": "readme"
    }
]

In [34]:
def get_ghcn_specs(name):
    return [g for g in ghcn_specs if g["name"] == name][0]

In [51]:
def get_ghcn_file(file_name:str = "ghcnd-stations.txt"):
    file = ""
    with closing(FTP("ftp.ncdc.noaa.gov")) as ftp:
        ftp.login()
        with closing(StringIO()) as r:
            # need to add newlines, hence custom call back
            ftp.retrlines(
                "RETR /pub/data/ghcn/daily/{file_name}".format(file_name = file_name),
                callback = lambda line: r.write(line + "\n"))
            file = r.getvalue()
    return file

In [52]:
stations = get_ghcn_file("ghcnd-stations.txt")

In [75]:
stations_field_pos = [(field["pos_from"] - 1, field["pos_to"]) for field in get_ghcn_specs("stations")["field_specs"]]

In [76]:
stations_field_names = [field["field_name"] for field in get_ghcn_specs("stations")["field_specs"]]

In [79]:
stations_df = pd.read_fwf(
    StringIO(stations),
    colspecs = stations_field_pos,
    header = None)

stations_df.columns = stations_field_names

In [80]:
stations_df.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0


In [81]:
len(stations_df)

114789

In [82]:
stations_df.STATE.value_counts()

TX    5002
CO    4161
CA    2790
NC    2143
NE    2097
      ... 
MP      11
UM      11
PW      11
SA       1
PI       1
Name: STATE, Length: 76, dtype: int64

In [84]:
stations_df[stations_df.STATE == "NY"]

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
75400,US1NYAB0001,42.6670,-74.0509,445.0,NY,ALTAMONT 2.7 SSW,NaN,NaN,NaN
75401,US1NYAB0006,42.7198,-73.9304,88.7,NY,SCHENECTADY 5.6 SSW,NaN,NaN,NaN
75402,US1NYAB0010,42.5455,-74.1475,488.3,NY,RENSSELAERVILLE 2.1 NNW,NaN,NaN,NaN
75403,US1NYAB0016,42.7590,-73.7370,104.2,NY,LATHAM 1.1 NNE,NaN,NaN,NaN
75404,US1NYAB0017,42.6678,-73.7856,65.5,NY,ALBANY 0.7 E,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
113936,USW00094740,44.3853,-74.2067,506.9,NY,SARANAC RGNL AP,NaN,NaN,NaN
113938,USW00094745,41.0669,-73.7075,115.5,NY,WESTCHESTER CO AP,NaN,NaN,NaN
113941,USW00094789,40.6386,-73.7622,3.4,NY,NEW YORK JFK INTL AP,NaN,NaN,74486.0
113942,USW00094790,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP,NaN,NaN,NaN
